# Field Project Cartesius Capital: trend following

- Daniel de Miranda Almeida
- Kauan Mariani Ferreira
- Leonardo Alexandre da Silva Ferreira
- Pedro Henrique Coterli

Este relatório apresenta a metodologia, o processo e os resultados obtidos do projeto de campo realizado pelos alunos da FGV citados acima em conjunto com a Cartesius Capital.

## O problema

A Cartesius Capital é uma fundtech que faz a gestão de fundos multimercados usando sistemas de inteligência artificial preditiva. Sendo assim, utiliza de diversos modelos de aprendizado de máquina (machine learning) que geralmente necessitam de grandes bases de dados para a obtenção de relativa eficiência. Entretanto, há ativos que não apresentam histórico suficiente, em geral por terem iniciado recentemente.

Nosso problema era tentar criar uma abordagem para lidar com esses ativos, utilizando de técnicas de perseguição de tendência (trend following) na tentativa de realizar a geração de rótulos de transações (trades) com base nos poucos dados disponíveis.

## A abordagem

A abordagem criada por nós foi inspirada nas técnicas de **bagging** de machine learning, em que vários modelos medianos são utilizados conjuntamente na tentativa de gerarem uma predição de qualidade mais elevada.

### Os dados

Inicialmente, coletamos dos dados uma **lista de indicadores** relacionados a trend following e salvamos tudo isso em um DataFrame. Esses são os dados sobre os quais iremos trabalhar. Para isso, utilizamos a biblioteca [`ta`](https://technical-analysis-library-in-python.readthedocs.io/en/latest/), que já possui a implementação desses indicadores. Após esse cálculo, apenas realizamos a normalização dos novos dados. Esse processo pode ser encontrado no módulo [`indicators.py`](./indicadores.py).

### A rotulagem

Em seguida, precisamos fazer a rotulagem dos dados de treinamento, indicando para cada dia do ativo, com base nos dados iniciais de OHLC (Open, High, Low, Close), a ação ideal (compra, venda ou neutralidade). Para isso, utilizamos do **método de barreira tripla**, que está implementado no módulo [`labeling.py`](./labeling.py).

### O modelo

A próxima etapa compreende a seleção de um modelo de machine learning e seu treinamento com os dados e rótulos gerados anteriormente. Com o uso da biblioteca `sklearn`, optamos pelo emprego de um modelo de **random forest** para a predição, tanto por possuir um desempenho relativamente eficiente quanto por geralmente apresentar resultados mais precisos. Com isso, selecionamos um ano de dados para ser separado para o processo de backtest e utilizamos o restante para o treinamento do modelo. Essas etapas estão codadas nos módulos [`backtesting_process.py`](./backtesting_process.py) e [`main_process.py`](./main_process.py).

Por fim, utilizamos a biblioteca [`backtesting`](https://pypi.org/project/Backtesting/) para simular a estratégia prevista para o ano de backtest e visualizar seus resultados. Para isso, temos no módulo [`tradingUtils.py`](./tradingUtils.py) a implementação de uma classe base dessa estratégia na forma exigida pela biblioteca.

### A tunagem

Por último, dado que temos uma grande quantidade de hiperparâmetros variáveis, principalmente por parte dos próprios indicadores, utilizamos a biblioteca [`optuna`](https://optuna.org) para realizar sua tunagem. Para isso, implementamos no módulo [`tuner.py`](./tuner.py) esse processo, em que a biblioteca toma os diversos hiperparâmetros com seus respectivos intervalos de experimentação e realiza um processo de testes sucessivos sempre gerando novamente os dados de indicadores, treinando o modelo e observando seu desempenho no ano separado para validação (que deve ser diferente do ano de backtest). Finalizado isso, os melhores valores de hiperparâmetros são salvos em arquivos JSON na pasta [`hyperparams`](./hyperparams/).